# Агрументы командной строки

Аргументы передаваемые при запуске интерепретатора python
- `sys.argv` - список аргументов командной строки, передаваемых сценарию Python.  
- `sys.argv[0]` является именем скрипта (пустой строкой в интерактивной оболочке).

In [1]:
import sys
sys.argv

['/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py',
 '-f',
 '/home/nbuser/.local/share/jupyter/runtime/kernel-01bd1eb6-255b-4f69-aeff-b63909acc31b.json']

Официально запуск вашего скрипта выглядит следующим образом:
`/path/to/python.exe /path/to/file.py`
Сначала идет команда, которую нужно выполнить, далее – аргументы, которые в нее подаются, например, путь до скрипта.

Аргументов может быть несколько.
`/path/to/python.exe /path/to/file.py –v –b –i “Test”`
Каждое слово (разделенное пробелом) – отдельный аргумент.

Как получить аргументы из командной строки?
Простой способ – обратиться к модулю `sys`, объекту `argv`.

`sys.argv` возвращает список аргументов командной строки, где каждый элемент список – отдельный аргумент (слово)

```
python.exe myfile.py –v 3 
sys.argv -> ['myfile.py', '-v', '3']
```

Попробуем сделать скрипт, который будет принимать один аргумент, который будет отвечать за имя. И на основании этого выдавать приветсвенное сообщение

In [ ]:
import sys

if __name__ == "__main__":
    if len(sys.argv) == 1:
        print("Привет, мир!")
    else:
        if len(sys.argv) < 3:
            print("Ошибка. Слишком мало параметров.")
            sys.exit(1)

        if len(sys.argv) > 3:
            print("Ошибка. Слишком много параметров.")
            sys.exit(1)

        param_name = sys.argv[1]
        param_value = sys.argv[2]

        if (param_name == "--name" or
                param_name == "-n"):
            print ("Привет, {}!".format (param_value) )
        else:
            print ("Ошибка. Неизвестный параметр '{}'".format (param_name) )
            sys.exit (1)

## argparse
`sys.argv` дает лишь возможность прочитать аргументы командной строки, проверять их корректность, наличие, выводить справку по ним приходится самим.
Для решения данных проблем было создано несколько модулей, один из них – `argparse`.

`argparse` включен в стандартный пакет интерпретатора, умеет работать с всеми видами аргументов (позиционные, именованные), поддерживает субкоманды и много еще чего.

``` python
import argparse

def createParser ():
    parser = argparse.ArgumentParser()
    # вопрос означает что аргумент должен быть один или его может не быть
    parser.add_argument('name', nargs='?')  # создадим один необязательный позиционный аргумент
    return parser
 

if __name__ == '__main__':
    parser = createParser()  # создаем парсер
    namespace = parser.parse_args()  # парсим аргументы
 
    print (namespace)  # в переменной namespace будут находиться ваши расперсеные аргументы
 
    if namespace.name:
        print ("Привет, {}!".format (namespace.name) )
    else:
        print ("Привет, мир!")
```


#### Парсинг аргументов
```python
parser.add_argument(‘-n’,
                    dest=‘n’, 
                    action=“store”, 
                    type=int, 
                    required=True, 
                    help=‘Simple input counter’)
```

Структура команды:
- `-n` – название аргумента, может начинаться с дефиса или нет
 - Если без дефиса – аргумент позиционный
 - Если с – аргумент именованный
- `dest` – название переменной, в которой будет сохранено значение аргумента (если отсутствует – берется название аргумента)
- `action` – действие, которое будет выполнено при наличии этого параметра.
- `store` – сохраняет значение аргумента 
- `store_true, store_false, store_const` – сохраняет `True`, `False` или заранее заданное константное значение аргумента
- `type` – тип данных значения
- `required` – обязателен ли аргумент
- `help` – подсказка, которая формируется и выводится в консоль

In [ ]:
# добавление значения по умолчанию
parser.add_argument('name', nargs='?', default='мир')

### Именнованные аргументы
``` python
parser.add_argument ('-n', '--name', default='мир')
```
Все именованные параметры считаются необязательными

Значение по умолчанию равно `None`

С помощью аргумента `required` именованные параметры можно сделать обязательным
``` python
parser.add_argument ('-n', '--name', required=True)
```

### Параметры как списки
``` python
parser.add_argument ('-n', '--name', nargs='+', default=['test'])
```

### Указание типов параметров
``` python
parser.add_argument ('-c', '--count', default=1, type=int)
```

### Имена файлов как параметры
``` python
parser.add_argument ('-n', '--name', type=open)
```

Но если использовать `type=open`, то можно получить пугающий листинг при вызове ошибки. Поэтому есть `argparse.FileType()`, который обработает ошибку и выдаст её в читабельном виде
``` python
parser.add_argument ('-n', '--name', type=argparse.FileType())
argparse.FileType(mode='r', bufsize=-1, encoding=None, errors=None)
```
Аналогично мы можем принимать имя файла для записи, при этом только надо не забыть передать в фукнцию `argparse.FileType` в качестве первого параметра строку `'w'`.

In [ ]:
как работает argparse.FileType

### Параметры как флаги
```python
parser.add_argument ('-g', '--goodbye', action='store_const', const=True)
parser.add_argument ('-g', '--goodbye', action='store_const', const=True, default=False)

parser.add_argument ('-g', '--goodbye', action='store_true', default=False)
```

### Subparsers 
Можно указывать различные подпарсеры, каждый из которых будет включаться только при определенных флагах и по-разному парсить дальнейший вход.
```
python network.py train -data /path/to/file.txt -classes /path/to/another/file.txt
python network.py test -input /path/to/test_file.txt
```
Причем, `-data` и `-classes` аргументы при флаге `train`, а `-input` аргумент при флаге `test`

``` python
subparsers = parser.add_subparsers(dest='command')

train_parser = subparsers.add_parser('train')
train_parser.add_argument('-data', dest='data', required=True)
train_parser.add_argument('-classes', dest='classes', required=True)

test_parser = subparsers.add_parser('test')
test_parser.add_argument('-input', required=True)
```

Ожидает обязательный именованный аргумент со строкой.
```python
parser.add_argument('-i', type=str, required=True)  
```

Ожидает именованный параметр `--user` и сохраняет в переменную значение `True`
```python
parser.add_argument('--user', action="store_true")
```